# 1. Retrieval Augmented In-context Learning

- Please choose "Runtime Type" = GPU in Colab for running this notebook (Runtime > Change Runtime Type > T4 GPU).
- You are free to choose to use Google Colab or Kaggle to complete this notebook.

## 1.1 Contextual Embedding

In [1]:
# Step 0. Prepare the environment
!pip install InstructorEmbedding sentence-transformers datasets scikit-learn backoff evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 37.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (

In [2]:
!mkdir -p data/classification
!wget -O data/classification/train.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/train.txt
!wget -O data/classification/dev.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/dev.txt
!wget -O data/classification/test-blind.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/test-blind.txt

--2024-04-27 13:52:21--  https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 738844 (722K) [text/plain]
Saving to: ‘data/classification/train.txt’

data/classification 100%[===================>] 721.53K  --.-KB/s    in 0.02s   

2024-04-27 13:52:21 (42.9 MB/s) - ‘data/classification/train.txt’ saved [738844/738844]

--2024-04-27 13:52:21--  https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/dev.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

In [3]:
# Step 1. Declare the model & Example usage
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("hkunlp/instructor-base")
embeddings = model.encode(
    [
        "Dynamical Scalar Degree of Freedom in Horava-Lifshitz Gravity",
        "Comparison of Atmospheric Neutrino Flux Calculations at Low Energies",
        "Fermion Bags in the Massive Gross-Neveu Model",
        "QCD corrections to Associated t-tbar-H production at the Tevatron",
    ],
    prompt="Represent the Medicine sentence for clustering: ",
    show_progress_bar=True,
)

print(embeddings.shape)

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

2_Dense/pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(4, 768)


In [4]:
# Step 2. Load data & Extract embeddings
# It may takes about two hours for CPU, or you can switch to GPU runtime for faster inference.
def load_data(file):
    r"""Load your custom data for training or evaluation.

    Args:
        file (str): the path of your data

    Returns:
        tuple: a tuple containing two elements:
           - embedding: The embedding of your data, should in shape (N, 768)
          - label: List of labels, should in shape (N,)
    """
    with open(file) as f:
        data = [line.strip().split('\t') for line in f]
    sentences = [line[1] for line in data]
    X = model.encode(sentences, prompt="Represent the sentence for sentiment analysis: ", show_progress_bar=True)
    y = [int(line[0]) for line in data]
    return X, y

X_train, y_train = load_data('data/classification/train.txt')
X_val, y_val = load_data('data/classification/dev.txt')

print(f"X_train shape: {X_train.shape}, y_train shape: {len(y_train)}")
print(f"X_val shape: {X_val.shape}, y_val shape: {len(y_val)}")

Batches:   0%|          | 0/217 [00:00<?, ?it/s]

Batches:   0%|          | 0/28 [00:00<?, ?it/s]

X_train shape: (6920, 768), y_train shape: 6920
X_val shape: (872, 768), y_val shape: 872


And train a logistic regression model

In [5]:
# Step 3: Train a logistic regression model
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=3361)
model.fit(X_train, y_train)

LogisticRegression(random_state=3361)

In [6]:
# Step 4: Evaluate the model
from sklearn.metrics import precision_score, recall_score, f1_score

y_pred = model.predict(X_val)

precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f"Precision = {precision}")
print(f"Recall = {recall}")
print(f"f1 = {f1}")

Precision = 0.8967032967032967
Recall = 0.918918918918919
f1 = 0.9076751946607342


| Metric    | Score (Contextual Embedding)  | Score (GloVe Embeddings, from A1) |
|-----------|:-----------------------------:|:------------------------:|
| Precision | 0.8967                        | 0.7709 |
| Recall    | 0.9189                        | 0.7883 |
| f1        | 0.9077                        | 0.7795 |

## 1.2 Retrieve Relevant Examples

In [7]:
import re
import json
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

def load_train_data():
    """Loads the GSM8k train dataset.

    Returns:
        A list of dictionaries containing questions, cot answers, and short answers.
    """
    ds = load_dataset("gsm8k", "main", split="train")
    examples = [{"question": example["question"], "answer": example["answer"]} for example in ds]
    for example in examples:
        example["short_answer"] = re.sub(r"(\d),(\d)", r"\1\2", example["answer"].split("####")[1].strip())
        example["cot_answer"] = re.sub(r"\<\<.*?\>\>", "", example["answer"].split("####")[0].strip()) \
            + " So the answer is " + example["short_answer"] + "."
    return examples

def load_test_data():
    """Loads the first 30 examples of the GSM8k test dataset.

    Returns:
        A list of dictionaries containing questions and answers.
    """
    ds = load_dataset("gsm8k", "main", split="test")
    examples = [{"question": example["question"], "answer": example["answer"].split("####")[1].strip()} for example in ds]
    for example in examples:
        example["answer"] = re.sub(r"(\d),(\d)", r"\1\2", example["answer"])
    return examples[:30]

In [8]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def retrieve_examples():
    """Retrieve top-20 in-context examples from GSM8K training set for each testing examples.

    Returns:
        A dictionary mapping testing questions to a list of top-20 training examples.
    """
    reference_set = load_train_data() # questions from the training set
    query_set = load_test_data() # questions from the test set

    reference_questions = [example["question"] for example in reference_set]
    query_questions = [example["question"] for example in query_set]

    # Step 1: embedding generation
    model = SentenceTransformer("hkunlp/instructor-base")
    reference_embeddings = model.encode(
        reference_questions,
        prompt="Represent the math question sentence for example retrieval: ",
        show_progress_bar=True,
    )
    query_embeddings = model.encode(
        query_questions,
        prompt="Represent the math question sentence for example retrieval: ",
        show_progress_bar=True,
    )

    # Step 2: similarity computation
    similarity_matrix = cosine_similarity(query_embeddings, reference_embeddings)

    # Step 3: Example retrieval
    retrieved_examples = []
    for i in range(similarity_matrix.shape[0]):
        example = {"query_example": query_set[i]}
        most_similar_examples = []
        # obtains the indices with the highest cosine similarity
        sorted_reference_idxs = np.argsort(similarity_matrix[i, :])[::-1][:20].tolist()
        for j in sorted_reference_idxs:
            most_similar_examples.append({"example": reference_set[j],
                                          "similarity": float(similarity_matrix[i][j])})
        example["similar_examples"] = most_similar_examples
        retrieved_examples.append(example)

    return retrieved_examples

In [9]:
RETRIEVED_EXAMPLES = retrieve_examples()

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Batches:   0%|          | 0/234 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Note: The following retrieval augmented generation does not require a GPU. Please consider saving and downloading the examples you retrieve from the left file tab so that you will not be hindered by Colab GPU restrictions.

In [10]:
with open("retrieved_examples.json", "w") as fout:
    json.dump(RETRIEVED_EXAMPLES, fout, indent=4)

with open("retrieved_examples.json", "r") as fin:
    RETRIEVED_EXAMPLES = json.load(fin)

## 1.3 Generation with Huggingface Inference API

We will use LLM by querying huggingface inference api so we do not need GPU for the following code. Please generate HF_TOKEN at [hf.co/settings/tokens](hf.co/settings/tokens) and set as environment variable.

In [11]:
from abc import ABC, abstractmethod
from typing import List, Dict, Any
import os
import json
import backoff
import evaluate
import re
from datetime import datetime
import time
import requests
import math
from google.colab import userdata

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# using Google Colab's secret manager to get the token
HF_TOKEN = userdata.get('HF_TOKEN')

# replace this line with the commented code to get the HF token from .env file
# HF_TOKEN = os.environ.get('HF_TOKEN')

from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

class LLM(object):
    def __init__(self, model_name="codellama/CodeLlama-7b-hf"):
        self.model_name = model_name
        self.api_url = f"https://api-inference.huggingface.co/models/{model_name}"
        self.headers = {"Authorization": f"Bearer {HF_TOKEN}"}

    @backoff.on_exception(backoff.expo, requests.exceptions.RequestException, max_time=60)
    def generate(self, prompts: List[str], **kwargs) -> List[str]:
        outputs = []

        def query(payload, retry: int=5, retry_secs: int = 30 * 60):
            for _ in range(retry):
                response = requests.post(self.api_url, headers=self.headers, json=payload)
                if response.status_code == 429:  # rate limit reached
                    print(f"{datetime.now()} Rate limit reached. Retry after {retry_secs / 60} minutes...")
                    time.sleep(retry_secs)
                    print(f"{datetime.now()} Retrying to call the API again...")
                elif response.status_code != 200:
                    raise ValueError(f"Request failed with code {response.status_code}, {response.text}")
                else:
                    return response.json()

        for prompt in prompts:
            data = query(
                {
                    "inputs": prompt,
                    "parameters": { "max_new_tokens": 256, "stop": ["Question:"]},
                }
            )

            outputs.append(data[0]['generated_text'])

        return outputs

In [12]:
llm = LLM("codellama/CodeLlama-7b-hf")
print(llm.generate(["Explain the importance of low latency LLMs", "What is the capital of France?"]))

['Explain the importance of low latency LLMs\n\n### Explain the importance of low latency LLMs\n\n-   Latency is the time it takes for a packet to travel from one point to another.\n\n-   Latency is the time it takes for a packet to travel from one point to another.\n\n-   Latency is the time it takes for a packet to travel from one point to another.\n\n-   Latency is the time it takes for a packet to travel from one point to another.\n\n-   Latency is the time it takes for a packet to travel from one point to another.\n\n-   Latency is the time it takes for a packet to travel from one point to another.\n\n-   Latency is the time it takes for a packet to travel from one point to another.\n\n-   Latency is the time it takes for a packet to travel from one point to another.\n\n-   Latency is the time it takes for a packet to travel from one point to another.\n\n-   Latency is the time it takes for a packet to travel from one point to another.\n\n-   Latency is the time it', 'What is the 

Please adapt your GSM8KCoTEvaluator for this API-based LLM. And report the performance of 8-shot chain-of-thought prompting on first 30 examples of GSM8K.

In [13]:
class Evaluator(ABC):
    def __init__(self, llm):
        self.llm = llm

    @abstractmethod
    def load_data(self):
        pass

    @abstractmethod
    def build_prompts(self):
        pass

    @abstractmethod
    def postprocess_output(self, output: str) -> str:
        pass

    def generate_completions(self, prompts: List[str], batch_size=2) -> List[str]:
        completions = []
        n_prompts = len(prompts)
        n_iter = math.ceil(n_prompts / batch_size)
        for i in tqdm(range(n_iter)):
            batch_prompts = prompts[batch_size*i : min(n_prompts, batch_size*(i+1))]
            batch_completions = self.llm.generate(batch_prompts)
            completions.extend(batch_completions)
        return completions

    def evaluate(self, batch_size=4, n_shot=8, save_dir="outputs"):
        dataset = self.load_data()
        prompts = self.build_prompts(dataset, n_shot)
        outputs = self.generate_completions(prompts, batch_size=batch_size)

        predictions = []
        for i, (example, prompt, output) in enumerate(zip(dataset, prompts, outputs)):
            prediction = {
                "task_id": example.get("task_id", f"task_{i}"),
                "prompt": prompt,
                "completion": output,
                "prediction": self.postprocess_output(output),
            }
            predictions.append(prediction)

        # Save predictions to file
        os.makedirs(save_dir, exist_ok=True)
        prediction_save_path = os.path.join(save_dir, f"{type(self).__name__}_predictions.jsonl")
        with open(prediction_save_path, "w") as fout:
            for pred in predictions:
                fout.write(json.dumps(pred) + "\n")

        # Calculate metrics and print results
        results = self.calculate_metrics(predictions, dataset)
        print(f"Results for {type(self).__name__}: {results}")

    @abstractmethod
    def calculate_metrics(self):
        pass

GSM_EXAMPLARS = [
    {
        "question": "There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?",
        "cot_answer": "There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. So the answer is 6.",
        "pot_answer": "def solution():\n    \"\"\"There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\"\"\"\n    trees_initial = 15\n    trees_after = 21\n    trees_added = trees_after - trees_initial\n    result = trees_added\n    return result",
        "short_answer": "6"
    },
    {
        "question": "If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?",
        "cot_answer": "There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. So the answer is 5.",
        "pot_answer": "def solution():\n    \"\"\"If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?\"\"\"\n    cars_initial = 3\n    cars_arrived = 2\n    total_cars = cars_initial + cars_arrived\n    result = total_cars\n    return result",
        "short_answer": "5"
    },
    {
        "question": "Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?",
        "cot_answer": "Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. So the answer is 39.",
        "pot_answer": "def solution():\n    \"\"\"Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?\"\"\"\n    leah_chocolates = 32\n    sister_chocolates = 42\n    total_chocolates = leah_chocolates + sister_chocolates\n    chocolates_eaten = 35\n    chocolates_left = total_chocolates - chocolates_eaten\n    result = chocolates_left\n    return result",
        "short_answer": "39"
    },
    {
        "question": "Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?",
        "cot_answer": "Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. So the answer is 8.",
        "pot_answer": "def solution():\n    \"\"\"Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?\"\"\"\n    jason_lollipops_initial = 20\n    jason_lollipops_after = 12\n    denny_lollipops = jason_lollipops_initial - jason_lollipops_after\n    result = denny_lollipops\n    return result",
        "short_answer": "8"
    },
    {
        "question": "Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?",
        "cot_answer": "Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. So the answer is 9.",
        "pot_answer": "def solution():\n    \"\"\"Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?\"\"\"\n    toys_initial = 5\n    mom_toys = 2\n    dad_toys = 2\n    total_received = mom_toys + dad_toys\n    total_toys = toys_initial + total_received\n    result = total_toys\n    return result",
        "short_answer": "9"
    },
    {
        "question": "There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?",
        "cot_answer": "There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. So the answer is 29.",
        "pot_answer": "def solution():\n    \"\"\"Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?\"\"\"\n    toys_initial = 5\n    mom_toys = 2\n    dad_toys = 2\n    total_received = mom_toys + dad_toys\n    total_toys = toys_initial + total_received\n    result = total_toys\n    return result",
        "short_answer": "29"
    },
    {
        "question": "Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?",
        "cot_answer": "Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. So the answer is 33.",
        "pot_answer": "def solution():\n    \"\"\"Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?\"\"\"\n    golf_balls_initial = 58\n    golf_balls_lost_tuesday = 23\n    golf_balls_lost_wednesday = 2\n    golf_balls_left = golf_balls_initial - golf_balls_lost_tuesday - golf_balls_lost_wednesday\n    result = golf_balls_left\n    return result",
        "short_answer": "33"
    },
    {
        "question": "Olivia has $23. She bought five bagels for $3 each. How much money does she have left?",
        "cot_answer": "Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8. So the answer is 8.",
        "pot_answer": "def solution():\n    \"\"\"Olivia has $23. She bought five bagels for $3 each. How much money does she have left?\"\"\"\n    money_initial = 23\n    bagels = 5\n    bagel_cost = 3\n    money_spent = bagels * bagel_cost\n    money_left = money_initial - money_spent\n    result = money_left\n    return result",
        "short_answer": "8"
    }
]

In [14]:
class GSM8KEvaluator(Evaluator):
    def load_data(self):
        ds = load_dataset("gsm8k", "main", split="test")
        examples = [{"question": example["question"], "answer": example["answer"].split("####")[1].strip()} for example in ds]
        for example in examples:
            example["answer"] = re.sub(r"(\d),(\d)", r"\1\2", example["answer"])
        return examples[:30]

    def build_prompts(self, dataset, n_shot=8, demos=GSM_EXAMPLARS):
        """
        Build few-shot prompts from the GSM8K dataset. Use
        :param dataset: list of examples
        :param n_shot: number of examples to use for few-shot learning
        :param demos: list of demonstrator examples
        :return: list of prompts
        """
        self.n_shot = n_shot

        prompts = []
        for example in dataset:
            prompt = 'Answer the following questions. '
            for examplar in demos[:n_shot]:
                prompt += f"\nQuestion: {examplar['question']} \nAnswer: {examplar['short_answer']}"
            question = example['question']
            prompt += f'\nQuestion: {question} \nAnswer: '
            prompts.append(prompt)
        return prompts

    def postprocess_output(self, output: str) -> str:
        """
        Postprocess the output from the language model.
        """
        return str(re.findall(r'Answer: (\d+,?\d*)', output)[self.n_shot]).replace(',', '')

    def calculate_metrics(self, predictions, dataset):
        """
        Calculate metrics for the GSM8K dataset
        """
        accuracy = 0
        assert len(predictions) == len(dataset), \
            f'found mismatch in prediction length: {len(predictions)} with no. of rows in dataset: {dataset.num_rows}'
        for i, (prediction, example) in tqdm(enumerate(zip(predictions, dataset))):
            predicted = prediction['prediction']
            ground_truth = example['answer']
            if predicted == ground_truth:
                accuracy += 1 / len(predictions)
        return accuracy

In [15]:
class GSM8KCoTEvaluator(GSM8KEvaluator):
    def build_prompts(self, dataset, n_shot=8, demos=GSM_EXAMPLARS):
        """
        Build few-shot prompts from the GSM8K dataset. Use
        :param dataset: list of examples
        :param n_shot: number of examples to use for few-shot learning
        :param demos: list of demonstrator examples
        :return: list of prompts
        """
        self.n_shot = n_shot
        prompts = []
        for example in dataset:
            prompt = 'Answer the following questions. Respond with "So the answer is ##" for the last sentence. '
            for examplar in demos[:n_shot]:
                prompt += f"\nQuestion: {examplar['question']} \nAnswer: {examplar['cot_answer']}"
            question = example['question']
            prompt += f'\nQuestion: {question} \nAnswer: '
            prompts.append(prompt)
        return prompts

    def postprocess_output(self, output: str) -> str:
        """
        Postprocess the output from the language model.
        """
        try:
            raw_number = str(re.findall(r'So the answer is[\d\+\-\*\/\= ]* [\$\-]?(\d+[\.,]?\d*)', output)[self.n_shot]).replace(',', '')
            processed_number = re.sub(r'\.$', '', raw_number)
            return processed_number
        except IndexError: # not following the usual format, will get wrong answer as no response detected
            return ''

In [16]:
cot_evaluator = GSM8KCoTEvaluator(llm)
cot_evaluator.evaluate(n_shot=8)

100%|██████████| 8/8 [01:38<00:00, 12.25s/it]
30it [00:00, 46551.65it/s]

Results for GSM8KCoTEvaluator: 0.13333333333333333


The accuracy for the first 30 examples in the test split is $\dfrac{4}{30}$, or $0.133$.

## 1.4 Impact of Quantity on Few-shot Prompting

In [17]:
cot_evaluator.evaluate(n_shot=1)
cot_evaluator.evaluate(n_shot=2)
cot_evaluator.evaluate(n_shot=4)

100%|██████████| 8/8 [01:03<00:00,  7.95s/it]
30it [00:00, 248183.67it/s]


Results for GSM8KCoTEvaluator: 0.03333333333333333


100%|██████████| 8/8 [01:02<00:00,  7.86s/it]
30it [00:00, 215830.39it/s]


Results for GSM8KCoTEvaluator: 0.06666666666666667


100%|██████████| 8/8 [01:22<00:00, 10.32s/it]
30it [00:00, 229615.18it/s]

Results for GSM8KCoTEvaluator: 0.06666666666666667


| n-shot | Accuracy |
|--------|----------|
| 1      | 0.033    |
| 2      | 0.067    |
| 4      | 0.067    |
| 8      | 0.133    |

## 1.5 Retrieval Augmented Few-shot Prompting

In [18]:
if os.path.exists("retrieved_examples.json"):
    with open("retrieved_examples.json", "r") as fin:
        RETRIEVED_EXAMPLES = json.load(fin)
else:
    raise FileNotFoundError("retrieved_example.json not found, please run 1.2 first")

In [19]:
class GSM8KRetrievalICLEvaluator(GSM8KEvaluator):
    def __init__(self, llm, ascending_similarity: bool = False):
        """Initiializer method.
        :param llm: The LLM to evaluate.
        :param bool ascending_similarity: Whether to sort the most similar questions in ascending order. Default: False.
        """
        self.llm = llm
        self.ascending = ascending_similarity

    def build_prompts(self, dataset, n_shot=8, demos=RETRIEVED_EXAMPLES,
                      ascending: bool = False):
        """Build prompts with RETRIVED_EXAMPLES we generated in 1.2.
        """
        self.n_shot = n_shot
        prompts = []
        for i, example in enumerate(dataset):
            prompt = 'Answer the following questions. Respond with "So the answer is ##" for the last sentence. '
            similar_examples = demos[i]["similar_examples"][:n_shot]
            if self.ascending:
                similar_examples = similar_examples[::-1]
            for examplar in similar_examples:
                prompt += f"\nQuestion: {examplar['example']['question']} \nAnswer: {examplar['example']['cot_answer']}"
            question = demos[i]["query_example"]["question"]
            prompt += f'\nQuestion: {question} \nAnswer: '
            prompts.append(prompt)
        return prompts

    def postprocess_output(self, output: str) -> str:
        """
        Postprocess the output from the language model.
        """
        try:
            raw_number = str(re.findall(r'So the answer is[\d\+\-\*\/\= ]* [\$\-]?(\d+[\.,]?\d*)', output)[self.n_shot]).replace(',', '')
            processed_number = re.sub(r'\.$', '', raw_number)
            return processed_number
        except IndexError: # not following the usual format, will get wrong answer as no response detected
            return ''

In [20]:
retrieval_icl_evaluator = GSM8KRetrievalICLEvaluator(llm)
retrieval_icl_evaluator.evaluate(n_shot=1)
retrieval_icl_evaluator.evaluate(n_shot=2)
retrieval_icl_evaluator.evaluate(n_shot=4)
retrieval_icl_evaluator.evaluate(n_shot=8)

100%|██████████| 8/8 [01:12<00:00,  9.06s/it]
30it [00:00, 223101.28it/s]


Results for GSM8KRetrievalICLEvaluator: 0.03333333333333333


100%|██████████| 8/8 [01:42<00:00, 12.84s/it]
30it [00:00, 240131.91it/s]


Results for GSM8KRetrievalICLEvaluator: 0.03333333333333333


100%|██████████| 8/8 [01:49<00:00, 13.74s/it]
30it [00:00, 72859.94it/s]


Results for GSM8KRetrievalICLEvaluator: 0.1


100%|██████████| 8/8 [02:06<00:00, 15.81s/it]
30it [00:00, 249166.57it/s]

Results for GSM8KRetrievalICLEvaluator: 0.2333333333333333


| n-shot | Accuracy |
|--------|----------|
| 1      | 0.033    |
| 2      | 0.033    |
| 4      | 0.100    |
| 8      | 0.233    |

In [21]:
# Rearrange the examples in ascending order of relevance.
retrieval_icl_evaluator_asc = GSM8KRetrievalICLEvaluator(llm, ascending_similarity=True)
retrieval_icl_evaluator_asc.evaluate(n_shot=1)
retrieval_icl_evaluator_asc.evaluate(n_shot=2)
retrieval_icl_evaluator_asc.evaluate(n_shot=4)
retrieval_icl_evaluator_asc.evaluate(n_shot=8)

100%|██████████| 8/8 [01:14<00:00,  9.30s/it]
30it [00:00, 281496.91it/s]


Results for GSM8KRetrievalICLEvaluator: 0


100%|██████████| 8/8 [01:38<00:00, 12.37s/it]
30it [00:00, 35696.20it/s]


Results for GSM8KRetrievalICLEvaluator: 0.06666666666666667


100%|██████████| 8/8 [02:02<00:00, 15.26s/it]
30it [00:00, 253177.30it/s]


Results for GSM8KRetrievalICLEvaluator: 0.1


100%|██████████| 8/8 [01:49<00:00, 13.72s/it]
30it [00:00, 50942.96it/s]

Results for GSM8KRetrievalICLEvaluator: 0.2333333333333333


| n-shot | Accuracy |
|--------|----------|
| 1      | 0.000    |
| 2      | 0.067    |
| 4      | 0.100    |
| 8      | 0.233    |

## Discussion

- Q1.1: Regarding the contextual embedding in Section 1.1, how does sentiment classification performance compare to your word2vec results in A1? Discuss potential reasons.

The sentiment classification performance using contextual embedding is significantly better than the performance using word2vec (GloVe vectors) results in Assignment 1.

One of the reasons is that the contextual embeddings captures more semantic meanings than word2vec embeddings. As we have prompted the `SentenceTransformer` model to generate sentence embeddings for a specific task (in this case, the embeddings are used to perform sentiment analysis), the embeddings can capture semantics that is specific to the downstream task that is currently performed. Moreover, the contextual embeddings are specific to a designated task, aiming to capture specific word instances of the embeddings. Thus, this increases the precision, recall and f1-score of the `LogisticRegression` model.

Another reason is that the contextual embeddings are dynamic while the word2vec word embeddings are static. This means that sentences with the same word can have different embeddings given its neighbouring context. For example, sentiment analysis often requires the capturing of negation words, such as "no" and "not". The adjectives followed by these words is expected to have a different semantic meaning compared with its word embedding alone. In this case, word2vec word embeddings fail to capture these semantics as the sentence vector is obtained by aggregating the embeddings of the individual words inside a sentence. On the other hand, individual words and sentences will have different contextual embeddings given a different context, which improves the performance of sentiment analysis as the dynamic embeddings can capture more semantics in each sentence.

- Q1.2: In Section 1.4, which discusses the impact of quantity, what trends do you notice when adding contextual examples? Do you think this trend will continue?

A trend that I observe is that the accuracy of the LLM answers increase when more contextual examples are added. As more examples are included, it is expected that this trend will continue as there will be a higher chance that at least one example is closely related to the query example. As the query example may become more relevant in terms of sentence structure and similarity, the chance that the attention layers in the LLMs attending to question-specific tokens may increase. Therefore, I believe that this trend will continue as expected.

- Q1.3: In Section 1.5, which covers retrieval-augmented in-context learning, how does this differ from Section 1.4? Analyze the reasons.

In this section, retrieval-augmented in-context learning is adopted to evaluate the performance of the LLM.

This method makes use of both in-context learning and retrieval augmentation to generate the few-shot prompts. Different from Section 1.4, which uses the same set of examples in generating the prompts, this section uses the top `n_shot` $\in \{1, 2, 4, 8\}$ example(s) that is the most similar to the evaluation question, as the set of few-shot examples to be included in the prompt. Retrieval augmentation is performed as the most similar examples from the GSM8K training set of questions are obtained for each test question, which improves the quality of the prompt.

Compared to the 8-shot performance in Section 1.4, the accuracy in Section 1.5 significantly improved from $0.1$ to $0.23$. This is because more examples that are relevant and similar to the test question are retrieved to form the LLM prompt. As a result, the LLM can attend better to previous prompts and tokens to generate a more desirable result.

- Q1.4: In Section 1.5, which arrangement yields better performance: in-context examples organized in descending or ascending order of relevance? Discuss the scenario.

The in-context examples organized in ascending order of relevance yields a better performance.

When the examples are organized in ascending order, the training question with the most similar contextual embeddings is arranged just before the test question. As a result, the LLM can pay more attention to the more recent tokens compared to previous tokens, thus generating completions based on more recent tokens, as these tokens are being assigned with higher attention weights. On the other hand, if the examples are organized in descending order, the training question with the most similar contextual embeddings is arranged as the first question, where the LLM may not be trained to assign high attention weights to far-away tokens. Thus, in some cases (the 2-shot and 8-shot instances), the accuracy is slightly higher when the in-context examples are organized in ascending order of relevance.

# 2. Basic Decoding Algorithms

In [22]:
!pip install transformers
!pip install datasets
!pip install evaluate

In [23]:
"""set device and random seeds"""

######################################################
#  The following helper functions are given to you.
######################################################

from tqdm.notebook import tqdm
import torch
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')

def set_seed(seed=19260817):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

device: cuda


In [24]:
"""load datasets"""

######################################################
#  The following helper code is given to you.
######################################################

from datasets import load_dataset

dataset = load_dataset('Ximing/ROCStories')
train_data, dev_data, test_data = dataset['train'], dataset['validation'], dataset['test']

print(train_data[0])

Generating train split:   0%|          | 0/16207 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1817 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1819 [00:00<?, ? examples/s]

{'story_id': '080198fc-d0e7-42b3-8e63-b2144e59d816', 'prompt': 'On my way to work I stopped to get some coffee.', 'continuation': 'I went through the drive through and placed my order. I paid the cashier and patiently waited for my drink. When she handed me the drink, the lid came off and spilled on me. The coffee hurt and I had to go home and change clothes.', 'constraint_words': ['drive', 'order', 'drink', 'lid', 'coffee', 'hurt', 'home', 'change', 'clothes']}


In [25]:
"""prepare evaluation"""

######################################################
#  The following helper code is given to you.
######################################################

from evaluate import load
from transformers import RobertaForSequenceClassification, RobertaTokenizer

perplexity_scorer = load("perplexity", module_type="metric")
cola_model_name = "textattack/roberta-base-CoLA"
cola_tokenizer = RobertaTokenizer.from_pretrained(cola_model_name)
cola_model = RobertaForSequenceClassification.from_pretrained(cola_model_name).to(device)

def batchify(data, batch_size):
    assert batch_size > 0

    batch = []
    for item in data:
        # Yield next batch
        if len(batch) == batch_size:
            yield batch
            batch = []

        batch.append(item)

    # Yield last un-filled batch
    if len(batch) != 0:
        yield batch

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at textattack/roberta-base-CoLA were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
"""set up evaluation metric"""

######################################################
#  The following helper code is given to you.
######################################################

def compute_perplexity(texts, model='gpt2', batch_size=8):
    score = perplexity_scorer.compute(predictions=texts, add_start_token=True, batch_size=batch_size, model_id=model)
    return score['mean_perplexity']


def compute_fluency(texts, batch_size=8):
    scores = []
    for b_texts in batchify(texts, batch_size):
      inputs = cola_tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)
      with torch.no_grad():
        logits = cola_model(**inputs).logits
        probs = logits.softmax(dim=-1)
        scores.extend(probs[:, 1].tolist())
    return sum(scores) / len(scores)


def compute_diversity(texts):
    unigrams, bigrams, trigrams = [], [], []
    total_words = 0
    for gen in texts:
        o = gen.split(' ')
        total_words += len(o)
        for i in range(len(o)):
            unigrams.append(o[i])
        for i in range(len(o) - 1):
            bigrams.append(o[i] + '_' + o[i + 1])
        for i in range(len(o) - 2):
            trigrams.append(o[i] + '_' + o[i + 1] + '_' + o[i + 2])
    return len(set(unigrams)) / len(unigrams), len(set(bigrams)) / len(bigrams), len(set(trigrams)) / len(trigrams)


def evaluate(generations, experiment):
    generations = [_ for _ in generations if _ != '']
    perplexity = compute_perplexity(generations)
    fluency = compute_fluency(generations)
    diversity = compute_diversity(generations)
    print(experiment)
    print(f'perplexity = {perplexity:.2f}')
    print(f'fluency = {fluency:.2f}')
    print(f'diversity = {diversity[0]:.2f}, {diversity[1]:.2f}, {diversity[2]:.2f}')
    print()

debug_sents = ["This restaurant is awesome", "My dog is cute and I love it.", "Today is sunny."]
evaluate(debug_sents, 'debugging run')

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

debugging run
perplexity = 178.64
fluency = 0.98
diversity = 0.87, 1.00, 1.00



In [27]:
"""load model and tokenizer"""

######################################################
#  The following helper code is given to you.
######################################################

from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name, pad_token="<|endoftext|>")
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In this section, you will implement a few basic decoding algorithms:
1. Greedy decoding
2. Vanilla sampling
3. Temperature sampling
5. Top-p sampling

We have provided a wrapper function `decode()` that takes care of batching, controlling max length, and handling the EOS token.
You will be asked to implement the core function of each method: *given the pre-softmax logits of the next token, decide what the next token is.*

**The wrapper calls the core function of each decoding algorithm, which you will implement in the subsections below.**

In [28]:
"""decode main wrapper function"""

######################################################
#  The following helper code is given to you.
######################################################

def decode(prompts, max_len, method, **kwargs):
    encodings_dict = tokenizer(prompts, return_tensors="pt", padding=True)
    input_ids = encodings_dict['input_ids'].to(device)
    attention_mask = encodings_dict['attention_mask'].to(device)

    model_kwargs = {'attention_mask': attention_mask}
    batch_size, input_seq_len = input_ids.shape

    unfinished_sequences = torch.ones(batch_size, dtype=torch.long, device=device)

    for step in range(max_len):
        model_inputs = model.prepare_inputs_for_generation(input_ids, **model_kwargs)
        with torch.no_grad():
            outputs = model(**model_inputs, return_dict=True, output_attentions=False, output_hidden_states=False)

        if step == 0:
            last_non_masked_idx = torch.sum(attention_mask, dim=1) - 1
            next_token_logits = outputs.logits[range(batch_size), last_non_masked_idx, :]
        else:
            next_token_logits = outputs.logits[:, -1, :]

        log_prob = F.log_softmax(next_token_logits, dim=-1)

        if method == 'greedy':
            next_tokens = greedy(next_token_logits)
        elif method == 'sample':
            next_tokens = sample(next_token_logits)
        elif method == 'temperature':
            next_tokens = temperature(next_token_logits, t=kwargs.get('t', 0.8))
        elif method == 'topk':
            next_tokens = topk(next_token_logits, k=kwargs.get('k', 20))
        elif method == 'topp':
            next_tokens = topp(next_token_logits, p=kwargs.get('p', 0.7))

        # finished sentences should have their next token be a padding token
        next_tokens = next_tokens * unfinished_sequences + tokenizer.pad_token_id * (1 - unfinished_sequences)

        input_ids = torch.cat([input_ids, next_tokens[:, None]], dim=-1)
        model_kwargs = model._update_model_kwargs_for_generation(outputs, model_kwargs, is_encoder_decoder=model.config.is_encoder_decoder)

        # if eos_token was found in one sentence, set sentence to finished
        unfinished_sequences = unfinished_sequences.mul((next_tokens != tokenizer.eos_token_id).long())

        if unfinished_sequences.max() == 0:
            break

    response_ids = input_ids[:, input_seq_len:]
    response_text = [tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True) for output in response_ids]

    return response_text

In [29]:
"""debug helper code"""

######################################################
#  The following helper code is given to you.
######################################################

# For debugging, we duplicate a single prompt 10 times so that we obtain 10 generations for the same prompt
dev_prompts = [dev_data[0]['prompt']] * 10

def print_generations(prompts, generations):
    for prompt, generation in zip(prompts, generations):
        print(f'{[prompt]} ==> {[generation]}')

## 2.1 Greedy Decoding

In [30]:
def greedy(next_token_logits):
    '''
    inputs:
    - next_token_logits: Tensor(size = (B, V), dtype = float)
    outputs:
    - next_tokens: Tensor(size = (B), dtype = long)
    '''

    # TODO: compute `next_tokens` from `next_token_logits`.
    # Hint: use torch.argmax()
    next_tokens = torch.argmax(next_token_logits, dim=1)

    return next_tokens


generations = decode(dev_prompts, max_len=20, method='greedy')
print_generations(dev_prompts, generations)

['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m goin

## 2.2 Vanilla Sampling and Temperature Sampling

In [31]:
def sample(next_token_logits):
    '''
    inputs:
    - next_token_logits: Tensor(size = (B, V), dtype = float)
    outputs:
    - next_tokens: Tensor(size = (B), dtype = long)
    '''

    # TODO: compute the probabilities `probs` from the logits.
    # Hint: `probs` should have size (B, V)
    f = torch.nn.Softmax(dim=1)
    probs = f(next_token_logits)

    # TODO: compute `next_tokens` from `probs`.
    # Hint: use torch.multinomial()
    next_tokens = torch.squeeze(torch.multinomial(probs, 1))

    return next_tokens


set_seed()
generations = decode(dev_prompts, max_len=20, method='sample')
print_generations(dev_prompts, generations)

['Ryan was called by his friend to skip work one day.'] ==> ['\n\nKiddiak said he stepped on it, hit the brakes hard and decided to cross first']
['Ryan was called by his friend to skip work one day.'] ==> [" Their interaction didn't work at all by the time he would get back to Louisville. Some details that"]
['Ryan was called by his friend to skip work one day.'] ==> [' According to investigators at the time, Gianfranco was yelling at them for not doing enough work']
['Ryan was called by his friend to skip work one day.'] ==> [' "No, my boyfriend doesn\'t do work," Cat started, desperate to dial a phone in distress']
['Ryan was called by his friend to skip work one day.'] ==> [' Later, he did not show up for school again until late the next day. After catching a train']
['Ryan was called by his friend to skip work one day.'] ==> [' The fliers denounced Gov. Pat Brown for having insufficient police presence in the building, and he engineered']
['Ryan was called by his friend to skip wo

In [32]:
def temperature(next_token_logits, t):
    '''
    inputs:
    - next_token_logits: Tensor(size = (B, V), dtype = float)
    - t: float
    outputs:
    - next_tokens: Tensor(size = (B), dtype = long)
    '''
    if t <= 0:
        raise ValueError("Invalid temperature t, the value of t should be strictly positive")

    # TODO: compute the probabilities `probs` from the logits, with temperature applied.
    f = torch.nn.Softmax(dim=1)
    probs = f(next_token_logits / t)

    # TODO: compute `next_tokens` from `probs`.
    next_tokens = torch.squeeze(torch.multinomial(probs, 1))

    return next_tokens


set_seed()
generations = decode(dev_prompts, max_len=20, method='temperature', t=0.8)
print_generations(dev_prompts, generations)

['Ryan was called by his friend to skip work one day.'] ==> ['\n\nKiddiak said he stepped on the blood spur out of pity from his cousin Jimmie']
['Ryan was called by his friend to skip work one day.'] ==> [" Their interaction didn't go well.\n\nHerad's mother was in a panic. Unable to"]
['Ryan was called by his friend to skip work one day.'] ==> [" He said he hadn't asked if he'd be coming back to work for a week.\n\n"]
['Ryan was called by his friend to skip work one day.'] ==> [' "I told my boss I was leaving work early to take care of my sons\' homework. So']
['Ryan was called by his friend to skip work one day.'] ==> [' He also said he was going to play out of town because the game was cancelled. Johnson also said']
['Ryan was called by his friend to skip work one day.'] ==> [' The next morning, after talking to other parents who also work in the area, he was told to']
['Ryan was called by his friend to skip work one day.'] ==> [' It was before he was going to college.\n\n"Oh, I\'l

## 2.3 Top-p Sampling

In [33]:
def topp(next_token_logits, p):
    '''
    inputs:
    - next_token_logits: Tensor(size = (B, V), dtype = float)
    - p: float
    outputs:
    - next_tokens: Tensor(size = (B), dtype = long)
    '''
    if p < 0 or p > 1:
        raise ValueError("Invalid p, the value of p should be in the range [0, 1]")

    # TODO: Sort the logits in descending order, and compute
    # the cumulative probabilities `cum_probs` on the sorted logits
    sorted_logits, sorted_indices = torch.sort(next_token_logits, descending=True)
    f = torch.nn.Softmax(dim=1)
    sorted_probs = f(sorted_logits)
    cum_probs = torch.cumsum(sorted_probs, dim=1)

    # Create a mask to zero out all logits not in top-p
    sorted_indices_to_remove = cum_probs > p
    sorted_indices_to_remove[:, 1:] = sorted_indices_to_remove[:, :-1].clone()
    sorted_indices_to_remove[:, 0] = 0
    # Restore mask to original indices
    indices_to_remove = sorted_indices_to_remove.scatter(dim=1, index=sorted_indices, src=sorted_indices_to_remove)

    # Mask the logits
    next_token_logits[indices_to_remove] = float('-inf')

    # TODO: Sample from the masked logits
    f = torch.nn.Softmax(dim=1)
    probs = f(next_token_logits)
    next_tokens = torch.squeeze(torch.multinomial(probs, 1))

    return next_tokens


set_seed()
generations = decode(dev_prompts, max_len=20, method='topp', p=0.7)
print_generations(dev_prompts, generations)

['Ryan was called by his friend to skip work one day.'] ==> ["\n\nKiddie said he also didn't know if he was working out or not, and"]
['Ryan was called by his friend to skip work one day.'] ==> [' On the day of the attack, he took the opportunity to share a beautiful moment with her, a']
['Ryan was called by his friend to skip work one day.'] ==> [' When he arrived at work, the new tenant was coming in with a pile of trash.\n\n']
['Ryan was called by his friend to skip work one day.'] ==> [' "No, my boyfriend doesn\'t do that," he said, before jogging out to his room']
['Ryan was called by his friend to skip work one day.'] ==> [' He also sat in a wheelchair at his hospital bed. In the room was a young woman who looked']
['Ryan was called by his friend to skip work one day.'] ==> [" The rookie's parents said he was talking to his father and asking him to help him.\n\n"]
['Ryan was called by his friend to skip work one day.'] ==> [" Then he goes back to work. He can't afford to do it. S

## 2.4: Evaluation

Run the following cell to obtain the evaluation results, which you should include in your writeup.
Also don't forget to answer the questions.

In [34]:
prompts = [item['prompt'] for item in test_data][:10]
GENERATIONS_PER_PROMPT = 10
MAX_LEN = 100

for experiment in ['greedy', 'sample', 'temperature', 'topp']:
    generations = []
    for prompt in tqdm(prompts):
        generations += decode([prompt] * GENERATIONS_PER_PROMPT, max_len=MAX_LEN, method=experiment)
    evaluate(generations, experiment)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

greedy
perplexity = 2.08
fluency = 0.78
diversity = 0.01, 0.02, 0.03



  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

...

  0%|          | 0/13 [00:00<?, ?it/s]

temperature
perplexity = 16.61
fluency = 0.68
diversity = 0.33, 0.79, 0.95



  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

topp
perplexity = 12.13
fluency = 0.74
diversity = 0.29, 0.75, 0.95



## Discussion
- Q2.1: In greedy decoding, what do you observe when generating 10 times from the test prompt?

In greedy decoding, it is observed that the 10 generations from the test prompt are the same. As greedy decoding directly obtains the next token as the token with the highest output probability, the LLM output is deterministic and therefore the generations each time are the same.

- Q2.2: In vanilla sampling, what do you observe when generating 10 times from the test prompt?

In vanilla sampling, it is observed that the 10 generations from the test prompt are different. As vanilla sampling obtains the next token from the output token distribution of the LLM, the output is random as the tokens are randomly sampled from the token distribution. Thus, the generations each time are different.

Moreover, as the tokens are randomly sampled, the stories generated may lack cohesion, coherence and logical sense, although the stories started to have more diversity and different sentence details. Specfically, there is one instance where the token "she" is generated, but at the start of the story, the main character is Ryan which is a man.

- Q2.3: In temperature sampling, play around with the value of temperature $t$. Which value of $t$ makes it equivalent to greedy decoding? Which value of $t$ makes it equivalent to vanilla sampling?

The temperatures $t \in \{0.01, 0.5, 0.99\}$ are experimented.

In [35]:
set_seed()
for t in [0.01, 0.5, 0.99]:
    generations = decode(dev_prompts, max_len=20, method='temperature', t=t)
    print(f'Temperature = {t}')
    print_generations(dev_prompts, generations)
    print('------------------\n')

Temperature = 0.01
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I wa

From the experiment above, it is observed that a low temperature produces a more deterministic story generations, and a higher temperature produces a more random story generations.

As the temperature is lowered, the logits of the LLM outputs are adjusted to become higher. As a result, the probability for the decoding algorithm to assign a high-probability token becomes even higher, which makes the result more deterministic.

As the temperature increases, the logits of the LLM outputs are adjusted to become lower. Consequently, the probability for the decoding algorithm to assign a high-probability token becomes lower, which makes the result more random.

As $t \rightarrow 0$, the temperature sampling becomes greedy decoding.

As $t \rightarrow \infty$, the temperature sampling becomes vanilla sampling.

- Q2.4: In top-$p$ sampling, play around with the value of $p$. Which value of $p$ makes it equivalent to greedy decoding? Which value of $p$ makes it equivalent to vanilla sampling?

The temperatures $p \in \{0, 0.5, 1\}$ are experimented.

In [36]:
set_seed()
for p in [0, 0.5, 1]:
    generations = decode(dev_prompts, max_len=20, method='topp', p=p)
    print(f'p = {p}')
    print_generations(dev_prompts, generations)
    print('------------------\n')

p = 0
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'

From the experiment above, it is observed that a low value of $p$ produces a more deterministic story generations, and a high value of $p$ produces a more random story generations.

As $p$ is lowered, the size of the token distribution decreases, meaning that the decoding algorithm can only sample from a smaller set of tokens. This makes the text completions less random and becoming more deterministic.

As $p$ increases, the size of the token distribution increases, meaning that the decoding algorithm can sample from a wider range of tokens. This makes the text completions more random and less predictable.

When $p = 0$, the top-$p$ sampling is equivalent to greedy decoding.

When $p = 1$, the top-$p$ sampling is equivalent to vanilla sampling.

- Q2.5: Report the evaluation metrics (perplexity, fluency, diversity) of all 4 decoding methods. Which methods have the best and worst perplexity? Fluency? Diversity?

The evaluation metrics of all 4 decoding methods are recorded as below.

|   Decoding Method   | Perplexity | Fluency | Diversity (Unigrams) | Diversity (Bigrams) | Diversity (Trigrams) |
|:-------------------:|:----------:|:-------:|:--------------------:|:-------------------:|:--------------------:|
|        Greedy       |    2.08    |   0.78  |         0.01         |         0.02        |         0.03         |
|   Vanilla sampling  |    70.58   |   0.34  |         0.43         |         0.90        |         0.99         |
| Temperature (`t=0.8`) |    16.61   |   0.68  |         0.33         |         0.79        |         0.95         |
|    Top_p (`p=0.7`)    |    12.13   |   0.74  |         0.29         |         0.75        |         0.95         |

In terms of perplexity, the greedy decoding method has the best (lowest) perplexity and the vanilla sampling method has the worst (highest) perplexity. As the LLM output becomes more deterministic (as in the case of greedy decoding), the effective vocabulary size of predicting the next token decreases, which lowers the perplexity.

In terms of fluency, the greedy decoding method has the best (highest) fluency and the vanilla sampling method has the worst (lowest) fluency. As the LLM output becomes less random (as in the case of greedy decoding), the tokens generated are with higher probability, which makes the sentences more grammatically acceptable and sensible. This increases the fluency.

In terms of diversity, the vanilla sampling method has the best (highest) diversity for different n-grams, and the greedy decoding method has the worst (lowest) diversity for all unigrams, bigrams and trigrams. As the LLM output becomes more random (as in the case of vanilla sampling), the tokens are generated more randomly. This results in more unique and distinct token patterns and sentence patterns generated. As diversity is the ratio of the unique $n$-grams scaled by the total $n$-grams of a sentences, a decoding algorithm with a less deterministic token generation method will yield a higher diversity.